In [73]:
%%capture --no-stderr
%pip install langchain langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community langchainhub

In [74]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [75]:
import bs4
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

loader = WebBaseLoader(
    web_paths=urls,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [76]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

In [77]:
retriever = vectorstore.as_retriever()
retrieved_docs = retriever.invoke("agent memory")

In [78]:
query = "agent_memory"

In [79]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

def format_docs(input_docs):
    return "\n\n".join(doc.page_content for doc in input_docs)

def check_relevance(user_query):
    relevance_prompt = PromptTemplate(
        template="사용자의 질문에 답변해줘. Context 를 추가로 제공할텐데 질문과 Context 가 연관이 있으면 yes, 연관이 없으면 no 라고 답변해줘. 응답 포맷은 아래와 같아.\n{format_instructions}\nQuestion: {question} \nContext: {context} \nAnswer:",
        input_variables=["question", "context"],
        partial_variables={"format_instructions": "{\"relevance\": \"yes\"} or {\"relevance\": \"no\"}"},
    )
    chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()}
            | relevance_prompt
            | llm
            | JsonOutputParser()
    )
    return chain.invoke(user_query)

In [80]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

def rag_result(user_query):
    rag_prompt = hub.pull("rlm/rag-prompt")
    rag_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()}
         | rag_prompt
         | llm
         | StrOutputParser()
         )
    return rag_chain.invoke(user_query)

In [81]:
def check_hallucination(llm_result):
    hallucination_prompt = PromptTemplate(
        template="내가 작성하는 내용에 거짓이 있는지 확인해줘. 거짓이 있다면 yes, 거짓이 없다면 no 라고 답변해줘 응답 포맷은 아래와 같아.\n{format_instructions}\nQuestion: {question} \nAnswer:",
        input_variables=["question"],
        partial_variables={"format_instructions": "{\"hallucination\": \"yes\"} or {\"hallucination\": \"no\"}"},
    )
    hallucination_chain = (
            {"question": RunnablePassthrough()}
            | hallucination_prompt
            | llm
            | JsonOutputParser()
    )
    return hallucination_chain.invoke(llm_result)

In [84]:
for i in [0, 1, 2]:
    relevance = check_relevance(query)
    if relevance['relevance'] == 'yes':
        print(relevance)
        x = rag_result(query)
        print(x)
        print(check_hallucination(x))
        break
    else:
        print(relevance)
        break

{'relevance': 'yes'}
Agent memory refers to the capability of autonomous agents, such as those powered by large language models, to retain and utilize past interactions to inform future behavior. This includes aspects like relationship memory, where agents can continue conversations or coordinate events based on prior exchanges. Such memory functionalities enhance the social dynamics and responsiveness of the agents in simulations.
{'hallucination': 'no'}
